This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [ ]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
from urllib.request import urlopen, Request

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [ ]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
from datetime import datetime
import json
afx_x_url = "https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}?start_date={START_DATE}&end_date={END_DATE}&api_key={READ_API_KEY}".format(database_code='FSE', dataset_code='AFX_X', return_format='json', START_DATE = datetime(2017, 1, 1).strftime('%Y-%m-%d') , END_DATE=(datetime(2017, 12, 31)).strftime('%Y-%m-%d'), READ_API_KEY=API_KEY)

request_afx_x = Request(url=afx_x_url, headers={'User-Agent': 'Mozilla/5.0'})   
client = urlopen(request_afx_x)
results_afx_x = client.read()
client.close()

# 2. Convert the returned JSON object into a Python dictionary.
payload_dict = json.loads(results_afx_x.decode('utf-8'))

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(payload_dict)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [ ]:
# helper method
def is_number(nbr: str) -> bool:
    if nbr is None:
        return False
    try:
        float(nbr)
        return True
    except ValueError:
        return False

In [ ]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

opening_price_all = [day_price[1] for day_price in payload_dict['dataset_data']['data'] if is_number(day_price[1])]
highest_price = max(opening_price_all)
lowest_price = min(opening_price_all)
print(f"Opening prices vary in interval [${lowest_price}...${highest_price}]")

In [ ]:
# 4. What was the largest change in any one day (based on High and Low price)?

largest_price_diff_day = max([abs(day_price[2] - day_price[3]) for day_price in payload_dict['dataset_data']['data'] if is_number(day_price[2]) and is_number(day_price[3])])
print(f"Largest change in any one day: ${largest_price_diff_day}")

In [ ]:
# 5. What was the largest change between any two days (based on Closing Price)?

closing_price_yesterday = 0.0
lagest_price_any_2_days = 0.0 
for index, day_price in enumerate(payload_dict['dataset_data']['data']):
    closing_price_today = day_price[4] 
    if closing_price_today is None:
        closing_price_today =  closing_price_yesterday # no change is missing data            
    elif index == 0:
        closing_price_yesterday =  closing_price_today # first day, set the yerterday's price as today's price 
    else:    
        price_diff = abs(closing_price_today - closing_price_yesterday)
        lagest_price_any_2_days = max(price_diff, lagest_price_any_2_days)

print(f"Largest change between any two days (based on Closing Price) ${lagest_price_any_2_days:>.2f}")   

In [ ]:
#6. What was the average daily trading volume during this year?

from functools import reduce
def avg (lst):
    return reduce(lambda a, b: a+b, lst) / len(lst)

trading_volume = [price_day[6] for price_day in payload_dict['dataset_data']['data']] 
print(f"Average daily trading volume is ${avg(trading_volume):>.2f}")   

In [ ]:
#7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

def median (lst_values):

    size_lst = len(lst_values)
    if size_lst % 2 == 0:
       return (lst_values[size_lst//2 - 1] + lst_values[size_lst//2]) / 2
    else:
        return lst_values[len(lst_values)//2 + 1] 


print(f"Median trading volume during this year ${median(sorted(trading_volume)):>.2f}")